
7.1  Create a crawler using the webcrawler provided and discussed in my jupyter notebook ( https://github.com/pschragger/big-data-python-class/blob/master/Lectures/Lecture_7_-_Link_Analysis.ipynb) , have it crawl the first 200 pages from a base URL of your choosing

In [ ]:
import scrapy
import os

In [3]:
mkdir code

In [4]:
cd code

C:\Users\alifa\OneDrive\Documents\Bigdata\Link_test\code


In [5]:
%%cmd
scrapy startproject tutorial

Microsoft Windows [Version 10.0.17134.345]
(c) 2018 Microsoft Corporation. All rights reserved.

(base) C:\Users\alifa\OneDrive\Documents\Bigdata\Link_test\code>scrapy startproject tutorial
New Scrapy project 'tutorial', using template directory 'C:\\ProgramData\\Anaconda2\\lib\\site-packages\\scrapy-1.5.1-py2.7.egg\\scrapy\\templates\\project', created in:
    C:\Users\alifa\OneDrive\Documents\Bigdata\Link_test\code\tutorial

You can start your first spider with:
    cd tutorial
    scrapy genspider example example.com

(base) C:\Users\alifa\OneDrive\Documents\Bigdata\Link_test\code>

In [6]:
cd tutorial

C:\Users\alifa\OneDrive\Documents\Bigdata\Link_test\code\tutorial


In [8]:
cd tutorial

C:\Users\alifa\OneDrive\Documents\Bigdata\Link_test\code\tutorial\tutorial


In [9]:
cd spiders

C:\Users\alifa\OneDrive\Documents\Bigdata\Link_test\code\tutorial\tutorial\spiders


In [45]:
%%writefile weather.py
import scrapy

class WeatherItem(scrapy.Item):
    title = scrapy.Field()
    link = scrapy.Field()
    resp = scrapy.Field()
    
class WeatherSpider(scrapy.Spider):
    name = 'weather'
    start_urls = ['https://weather.com/']

    #def parse(self, response):
        #hxs = scrapy.Selector(response)
       # titles = hxs.xpath('//ul/li')
        #item = []
       # for title in titles:
            #obj = WeatherItem()
          #  obj["title"] = title.xpath("a/text()").extract()
          #  obj["link"] = title.xpath("a/@href").extract()
         #   obj["resp"] = response
         #   if obj["title"] != []:
        #        item.append(obj)
     #   return item
    
    def parse(self, response):
        hxs = scrapy.Selector(response)
        titles = hxs.xpath('//ul/li')
        for i in range(0,200):
            for title in titles:
                yield{
                    'title': title.xpath("a/text()").extract(),
                    'link': title.xpath("a/@href").extract(),
                    'resp': response,
                    }
        if i<=200:
            yield scrapy.Request(
                response.urljoin(title.xpath("a/@href").extract()),
                callback=self.parse
            )
            
        

Overwriting weather.py


In [46]:
%%cmd
scrapy crawl weather -o weather_txt.csv -t csv

Microsoft Windows [Version 10.0.17134.345]
(c) 2018 Microsoft Corporation. All rights reserved.

(base) C:\Users\alifa\OneDrive\Documents\Bigdata\Link_test\code\tutorial\tutorial\spiders>scrapy crawl weather -o weather_txt.csv -t csv

(base) C:\Users\alifa\OneDrive\Documents\Bigdata\Link_test\code\tutorial\tutorial\spiders>

2018-11-12 18:48:16 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: tutorial)
2018-11-12 18:48:16 [scrapy.utils.log] INFO: Versions: lxml 4.2.1.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.19.0, Twisted 18.9.0, Python 2.7.15 |Anaconda, Inc.| (default, May  1 2018, 18:37:09) [MSC v.1500 64 bit (AMD64)], pyOpenSSL 18.0.0 (OpenSSL 1.0.2o  27 Mar 2018), cryptography 2.2.2, Platform Windows-10-10.0.17134
2018-11-12 18:48:16 [scrapy.crawler] INFO: Overridden settings: {'NEWSPIDER_MODULE': 'tutorial.spiders', 'FEED_URI': 'weather_txt.csv', 'SPIDER_MODULES': ['tutorial.spiders'], 'BOT_NAME': 'tutorial', 'ROBOTSTXT_OBEY': True, 'FEED_FORMAT': 'csv'}
2018-11-12 18:48:16 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2018-11-12 18:48:17 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloaderm

7.2  Create a Stochastic matrix from its resulting crawling as per https://cs7083.wordpress.com/2013/01/31/demystifying-the-pagerank-and-hits-algorithms/

In [74]:
import pandas as pd
import numpy as np

lst = []
raw = pd.read_csv("weather_txt.csv")
#raw=df.dropna()
raw['link'] = 'http://www.weather.com'+raw['link']
r = raw['resp'][0]
lst.append((r.split()[-1]).split('>')[-2])
for i in range(len(raw)):
    if raw['resp'][i] == r:
        lst.append(raw['link'][i])
    else:
        r = raw['resp'][i]
        lst.append((r.split()[-1]).split('>')[-2])
        
item = list(pd.DataFrame(lst)[0].unique())

link = []
length = len(lst)
for i, val in enumerate(lst):
    if i < length-1:
        link.append((lst[i], lst[i+1]))

n = pd.DataFrame(index=item, columns=item)
m = n.replace(np.NaN, 0)
for i in link:
    m.loc[i] = 1.0

ar = np.array(m)
v = ar.sum(axis=1)

result = ar/v[:, np.newaxis]
matrix = np.nan_to_num(result)

matrix

array([[0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.33333333, ..., 0.        , 0.        ,
        0.33333333],
       [0.        , 0.        , 0.33333333, ..., 0.        , 0.        ,
        0.33333333]])

7.3 Pass it through the Page Rank algorithm and provide the list of the top 5 page URLs in your sample

In [75]:
from numpy import *

def pagerank(H):
    n = len(H)
    w = zeros(n)
    rho = 1./n * ones(n);
    for i in range(n):
        if multiply.reduce(H[i]== zeros(n)):
            w[i] = 1
    newH = H + outer((1./n * w),ones(n))
 
    theta=0.85
    G = (theta * newH) + ((1-theta) * outer(1./n * ones(n), ones(n)))
    #print rho
    for j in range(10):
        rho = dot(rho,G)
        #print rho
    
    return list(rho)

In [76]:
import pandas as pd
import numpy as np

lst = []
raw = pd.read_csv("weather_txt.csv")
raw['link'] = 'http://www.weather.com'+raw['link']
r = raw['resp'][0]
lst.append((r.split()[-1]).split('>')[-2])
for i in range(len(raw)):
    if raw['resp'][i] == r:
        lst.append(raw['link'][i])
    else:
        r = raw['resp'][i]
        lst.append((r.split()[-1]).split('>')[-2])
        
item = list(pd.DataFrame(lst)[0].unique())

link = []
length = len(lst)
for i, val in enumerate(lst):
    if i < length-1:
        link.append((lst[i], lst[i+1]))

n = pd.DataFrame(index=item, columns=item)
m = n.replace(np.NaN, 0)
for i in link:
    m.loc[i] = 1.0

ar = np.array(m)
v = ar.sum(axis=1)

result = ar/v[:, np.newaxis]
matrix = np.nan_to_num(result)

matrix

array([[0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.33333333, ..., 0.        , 0.        ,
        0.33333333],
       [0.        , 0.        , 0.33333333, ..., 0.        , 0.        ,
        0.33333333]])

In [77]:
#please create matrix again after interpreting pagerank algorithm
r = pagerank(matrix)
value = ((pd.DataFrame(r)).sort_values(0, ascending=False)).head(5)
index = list(value.index)
index

[3L, 4L, 2L, 5L, 7L]

In [78]:
url = pd.DataFrame(item)
print "Top five URLs:"
for i in sort(index):
    print url[0][i]

Top five URLs:
http://www.weather.com/weather/today/l/USDC0001:1:US
http://www.weather.com/weather/hourbyhour/l/USDC0001:1:US
http://www.weather.com/weather/tenday/l/USDC0001:1:US
http://www.weather.com/weather/monthly/l/USDC0001:1:US
http://www.weather.com/life/manage-notifications


7.x For extra credit use the hits algorithm ( with a connectivity matrix )

In [79]:

def hits(A):
    n= len(A)
    Au= dot(transpose(A),A)
    Hu = dot(A,transpose(A))
    a = ones(n); h = ones(n)
    #print a,h
    for j in range(5):
        a = dot(a,Au)
        a= a/sum(a)
        h = dot(h,Hu)
        h = h/ sum(h)
    return h

In [80]:
value = pd.DataFrame(hits(ar))
sort = (value.sort_values(0, ascending=False)).head(5)
hit = list(sort.index)

In [85]:
import pandas as pd
url = pd.DataFrame(item)
print "Top 5 page URLs:"
for i in hit:
    print (url[0][i])

Top 5 page URLs:
nan
http://www.weather.com/accessibility
http://www.weather.com/forecast/ski/l/USDC0001:1:US
http://www.weather.com/tv/personalities
http://www.weather.com/weather.com
